In [1]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
from keras import backend as K
from keras.applications.vgg19 import VGG19
from keras.applications.vgg19 import preprocess_input
from keras.preprocessing import image

Using TensorFlow backend.


In [2]:
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import Dropout
K.set_image_data_format('channels_last')

In [3]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [5]:
df=pd.read_csv("C:/Users/jaina/Downloads/ESC-50-master/meta/esc50.csv")

In [6]:
processed_dir="C:/Users/jaina/Downloads/ESC-50-master/pro_dir_images/"

In [7]:
def create_dataset(img_folder):
   
    img_data_array=[]
    class_name=[]
   
    for file in os.listdir(img_folder):    
            image_path= os.path.join(img_folder, file)
            img = image.load_img(image_path, target_size=(224, 224))
            x = image.img_to_array(img)
            x = preprocess_input(x)
            img_data_array.append(x)
            cat=df[df["filename"].str[:-4]==file[:-4]].target.values[0]
            class_name.append(cat)
    return img_data_array, class_name

In [8]:
# extract the image array and class name
img_data, class_name =create_dataset(processed_dir)

In [9]:
X=np.array(img_data, np.float32)
y=np.array(class_name)

In [10]:
y = tf.keras.utils.to_categorical(y , num_classes=50)

In [11]:
model=keras.models.Sequential()
VGG = VGG19(include_top=False, input_shape=(224,224,3),pooling='avg')
model.add(VGG)
VGG.summary()
for layer in model.layers:
    layer.trainable=False
model.add(Dropout(rate=0.5))
model.add(Dense(216,activation='relu', kernel_regularizer='l2'))
model.add(Dense(108,activation='relu', kernel_regularizer='l2'))
model.add(Dense(90,activation='relu', kernel_regularizer='l2'))
model.add(Dense(50,activation='softmax'))

for layer in model.layers:
    print(layer.name, layer.trainable)

model.summary()


Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Model: "vgg19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_____________________________________________

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [13]:
from keras import optimizers
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(X_train,y_train, epochs=100,batch_size=None,validation_data=(X_test,y_test))
model.summary()


Train on 1600 samples, validate on 400 samples
Epoch 1/100
1600/1600 [==============================] - 192s 120ms/step - loss: 8.4378 - accuracy: 0.0444 - val_loss: 7.2859 - val_accuracy: 0.0825
Epoch 2/100
1600/1600 [==============================] - 157s 98ms/step - loss: 6.5318 - accuracy: 0.1412 - val_loss: 5.6422 - val_accuracy: 0.2375
Epoch 3/100
1600/1600 [==============================] - 158s 99ms/step - loss: 5.4091 - accuracy: 0.2081 - val_loss: 4.8258 - val_accuracy: 0.2650
Epoch 4/100
1600/1600 [==============================] - 158s 99ms/step - loss: 4.6435 - accuracy: 0.2806 - val_loss: 4.1570 - val_accuracy: 0.3625
Epoch 5/100
1600/1600 [==============================] - 160s 100ms/step - loss: 4.1168 - accuracy: 0.3469 - val_loss: 3.7695 - val_accuracy: 0.4000
Epoch 6/100
1600/1600 [==============================] - 158s 99ms/step - loss: 3.7721 - accuracy: 0.3681 - val_loss: 3.5301 - val_accuracy: 0.4525
Epoch 7/100
1600/1600 [==============================] - 159s 

1600/1600 [==============================] - 27s 17ms/step - loss: 1.9345 - accuracy: 0.6119 - val_loss: 2.1140 - val_accuracy: 0.5600
Epoch 55/100
1600/1600 [==============================] - 26s 16ms/step - loss: 1.9410 - accuracy: 0.6019 - val_loss: 1.9986 - val_accuracy: 0.6200
Epoch 56/100
1600/1600 [==============================] - 26s 16ms/step - loss: 1.9557 - accuracy: 0.6056 - val_loss: 2.0819 - val_accuracy: 0.5575
Epoch 57/100
1600/1600 [==============================] - 26s 16ms/step - loss: 1.9510 - accuracy: 0.6069 - val_loss: 2.0301 - val_accuracy: 0.5900
Epoch 58/100
1600/1600 [==============================] - 26s 16ms/step - loss: 1.9369 - accuracy: 0.6100 - val_loss: 1.9892 - val_accuracy: 0.6250
Epoch 59/100
1600/1600 [==============================] - 26s 16ms/step - loss: 1.9190 - accuracy: 0.5987 - val_loss: 2.0764 - val_accuracy: 0.5875
Epoch 60/100
1600/1600 [==============================] - 26s 16ms/step - loss: 1.9310 - accuracy: 0.6031 - val_loss: 2.0203 